In [14]:
from src.controller.HwpController import HwpController
from collections import OrderedDict

import re, uuid
import numpy as np

In [15]:
controller = HwpController()
path = '/Users/dobby/Desktop/Workspace/3월 계측보고서(신안산선2공구A-1).hwp'

controller.get_table_list(path, '일자별 발파 및 계측 현황')

unmatched field end


In [62]:
import re, uuid
import numpy as np

from src.service.ParseService import ParseService


class ComplicatedParser:
    def delete_non_target_data(self, table_data):
        """
        한글 표에서 원하지 않은 부분까지 나온 데이터를 정리하여 리스트로 반환합니다.
        target_data_text에 표에 반복적으로 들어가는 텍스트를 입력하여 필요없는 데이터를 삭제합니다.
        """
        target_data_text = ['일시', '발파', '진동', '소음', 'STA', '시간']

        target_data = [
            sublist for sublist in table_data
            if any(
                entry['row'] in ['0', '1'] and any(keyword in entry['text'] for keyword in target_data_text)
                for entry in sublist
            )
        ]
        
        target_data = [[item for item in items if item['text'] != ''] for items in target_data]

        return target_data

    def extract_columns(self, table_list):
        """
        한글 표에서 공통적인 컬럼 부분을 추출합니다.
        컬럼은 대부분 표의 시작 부분에 작성되기 때문에 row 값은 0 혹은 1에 위치하게 됩니다.
        이후 컬럼 값들이 딕셔너리로 저장되어 있기 때문에 중복을 제거 후 리스트로 반환합니다.
        """
        columns = []
        for items in table_list:
            for item in items:
                if item not in [i for i in columns]:
                    if int(item['row']) == 0 or int(item['row']) == 1:
                        columns.append(item)

        return columns
    
    def extract_non_column_data(self, table_list, columns):
        """
        한글 표에서 컬럼 부분을 제거한 나머지 데이터들을 반환합니다.
        """
        return [[item for item in items[len(columns):] if not int(item['colspan']) > 1]for items in table_list]


    def group_by_date(self, dict_list):
        """
        한글 표 데이터를 날짜 별로 분류하여 리스트로 저장하여 반환합니다.
        날짜 별 분류는 하나의 TableCell에 포함된 동일한 row값들 끼리 묶는 것으로 수행합니다.
        """
        group_list = []

        for items in dict_list:
            rows = list(set([int(item['row']) for item in items]))
            for row in rows:
                temp = [item for item in items if int(item['row'])==row]
                group_list.append(temp)
        
        return group_list 
    
    def update_merge_data(self, group_list):
        """
        한글 표에 병합 처리된 셀에 대한 데이터 처리를 완료한 뒤 리스트로 반환합니다.
        SimpleParser에서는 병합 셀이 아닌 셀의 최초 값이 공통 값으로 결정되기 때문에 이를 통해 값을 추가합니다.
        """
        cached_merge_data = []
        cached_head_data = []
        
        for idx, items in enumerate(group_list):
            head_data = [data for data in items if int(data['col']) == 0 and int(data['rowspan']) > 1]
            merge_data = [data for data in items if int(data['col']) != 0 and int(data['rowspan']) > 1]
            
            if head_data:
                cached_head_data = head_data
            if merge_data:
                cached_merge_data = merge_data

            new_items = []
            if any(data not in items for data in cached_head_data):
                new_items.extend(cached_head_data)
            if any(data not in items for data in cached_merge_data):
                new_items.extend(cached_merge_data)

            group_list[idx] = new_items + items
        
        return [items for items in group_list if len(items) > 1]
    
    def serialize_to_dict(self, group_list, columns):
        """
        컬럼 리스트와 파싱이 끝난 그룹 리스트를 이용해서 데이터를 분류한 뒤 리스트로 반환합니다.
        컬럼 리스트에 대응하는 값들을 그룹 리스트에서 찾아서 추가해주는 작업을 수행합니다.
        """
        serialize_list = []
        cached_columns = []
        
        for item in columns:
            if int(item['colspan']) > 1:
                for t in ['최저치', '최고치']:
                    data = {
                        'row': item['row'],
                        'col': item['col'],
                        'colspan': item['colspan'],
                        'rowspan': item['rowspan'],
                        'text': f'{item['text']} {t}'
                    }
                    cached_columns.append(data)
            else:
                cached_columns.append(item)

        for items in group_list:
            data = {column['text']: item['text'] for column, item in zip(cached_columns, items) if column != '발파횟수'}
            serialize_list.append(data)

        return serialize_list


In [63]:
service = ComplicatedParser()

In [64]:
table_list = service.delete_non_target_data(controller.get_list())

table_list

[[{'row': '0', 'col': '0', 'colspan': '1', 'rowspan': '2', 'text': '일시'},
  {'row': '0', 'col': '1', 'colspan': '1', 'rowspan': '2', 'text': '발파횟수'},
  {'row': '0', 'col': '2', 'colspan': '1', 'rowspan': '2', 'text': '시간'},
  {'row': '0', 'col': '3', 'colspan': '2', 'rowspan': '1', 'text': '장약량(kg)'},
  {'row': '0', 'col': '5', 'colspan': '1', 'rowspan': '1', 'text': '발파위치'},
  {'row': '0',
   'col': '6',
   'colspan': '4',
   'rowspan': '1',
   'text': '발파진동및소음측정치(max값)'},
  {'row': '1', 'col': '3', 'colspan': '1', 'rowspan': '1', 'text': '지발당장약량'},
  {'row': '1', 'col': '4', 'colspan': '1', 'rowspan': '1', 'text': '총장약량'},
  {'row': '1', 'col': '5', 'colspan': '1', 'rowspan': '1', 'text': 'STA'},
  {'row': '1',
   'col': '6',
   'colspan': '1',
   'rowspan': '1',
   'text': '발파진동(cm/s)'},
  {'row': '1',
   'col': '7',
   'colspan': '1',
   'rowspan': '1',
   'text': '진동레벨dB(V)'},
  {'row': '1',
   'col': '8',
   'colspan': '1',
   'rowspan': '1',
   'text': '소음레벨dB(A)'},
  {'row': '1

In [65]:
columns = service.extract_columns(table_list)

In [66]:
dict_list = service.extract_non_column_data(table_list, columns)

dict_list

[[{'row': '2', 'col': '0', 'colspan': '1', 'rowspan': '4', 'text': '3월1일'},
  {'row': '2', 'col': '1', 'colspan': '1', 'rowspan': '1', 'text': '1회'},
  {'row': '2', 'col': '2', 'colspan': '1', 'rowspan': '1', 'text': '7:01'},
  {'row': '2', 'col': '3', 'colspan': '1', 'rowspan': '1', 'text': '1.6-2.0'},
  {'row': '2', 'col': '4', 'colspan': '1', 'rowspan': '1', 'text': '300.0'},
  {'row': '2',
   'col': '5',
   'colspan': '1',
   'rowspan': '1',
   'text': '7k+771.25~774.25(종점)'},
  {'row': '2', 'col': '6', 'colspan': '1', 'rowspan': '1', 'text': 'N/T'},
  {'row': '2', 'col': '7', 'colspan': '1', 'rowspan': '1', 'text': 'N/T'},
  {'row': '2', 'col': '8', 'colspan': '1', 'rowspan': '1', 'text': 'N/T'},
  {'row': '2', 'col': '9', 'colspan': '1', 'rowspan': '1', 'text': '국원물산'},
  {'row': '3', 'col': '1', 'colspan': '1', 'rowspan': '2', 'text': '2회'},
  {'row': '3', 'col': '2', 'colspan': '1', 'rowspan': '2', 'text': '11:47'},
  {'row': '3', 'col': '3', 'colspan': '1', 'rowspan': '2', 'te

In [68]:
group_list = service.group_by_date(dict_list)

group_list

[[{'row': '2', 'col': '0', 'colspan': '1', 'rowspan': '4', 'text': '3월1일'},
  {'row': '2', 'col': '1', 'colspan': '1', 'rowspan': '1', 'text': '1회'},
  {'row': '2', 'col': '2', 'colspan': '1', 'rowspan': '1', 'text': '7:01'},
  {'row': '2', 'col': '3', 'colspan': '1', 'rowspan': '1', 'text': '1.6-2.0'},
  {'row': '2', 'col': '4', 'colspan': '1', 'rowspan': '1', 'text': '300.0'},
  {'row': '2',
   'col': '5',
   'colspan': '1',
   'rowspan': '1',
   'text': '7k+771.25~774.25(종점)'},
  {'row': '2', 'col': '6', 'colspan': '1', 'rowspan': '1', 'text': 'N/T'},
  {'row': '2', 'col': '7', 'colspan': '1', 'rowspan': '1', 'text': 'N/T'},
  {'row': '2', 'col': '8', 'colspan': '1', 'rowspan': '1', 'text': 'N/T'},
  {'row': '2', 'col': '9', 'colspan': '1', 'rowspan': '1', 'text': '국원물산'}],
 [{'row': '3', 'col': '1', 'colspan': '1', 'rowspan': '2', 'text': '2회'},
  {'row': '3', 'col': '2', 'colspan': '1', 'rowspan': '2', 'text': '11:47'},
  {'row': '3', 'col': '3', 'colspan': '1', 'rowspan': '2', 't

In [ ]:
group_list = service.update_merge_data(group_list)

group_list